## A / B Testing: Chi-2 with Montana Library case study

In this notebook we perform a Chi square test with data from the Library of Montana University case study, applying a post-hoc correction to perform pairwise tests and find the true winner.

Scipy approach.

### Data reading

The important pieces of information (clicks on each element of interest & visits on each page) are scattered around. Let's collect them:

In [1]:
import pandas as pd
import numpy as np
pd.set_option("max_colwidth", 1000)
pd.set_option("max_rows", 1000)

# Element list Homepage Version 1 - Interact, 5-29-2013.csv
url = 'https://drive.google.com/file/d/1Tj6Z4OtJqLBOW0z2fvuGS5EhZo8xTVM6/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
v1 = pd.read_csv(path)

# Element list Homepage Version 2 - Connect, 5-29-2013.csv
url = 'https://drive.google.com/file/d/1qHBdOjUWvJpN-LTg1z2jpeA3mDXQjdch/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
v2 = pd.read_csv(path)

# Element list Homepage Version 3 - Learn, 5-29-2013.csv
url = 'https://drive.google.com/file/d/1g8prRmy3hpVtL6zvkdCwXcgIV0CS48zr/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
v3 = pd.read_csv(path)

# Element list Homepage Version 4 - Help, 5-29-2013.csv
url = 'https://drive.google.com/file/d/1I9bjXkxtiILDogeQmsWCCDlQtRZ8OSrs/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
v4 = pd.read_csv(path)

# Element list Homepage Version 5 - Services, 5-29-2013.csv
url = 'https://drive.google.com/file/d/1noDp_jpdAL_LGxU3SPDxqP94pUCqisqW/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
v5 = pd.read_csv(path)

In [2]:
# clicks on each element
v1_clicks = int(v1.loc[v1["Name"]=="INTERACT"]["No. clicks"])
v2_clicks = int(v2.loc[v2["Name"]=="CONNECT"]["No. clicks"])
v3_clicks = int(v3.loc[v3["Name"]=="LEARN"]["No. clicks"])
v4_clicks = int(v4.loc[v4["Name"]=="HELP"]["No. clicks"])
v5_clicks = int(v5.loc[v5["Name"]=="SERVICES"]["No. clicks"])

In [3]:
print(v1_clicks, v2_clicks, v3_clicks, v4_clicks, v5_clicks)

42 53 21 38 45


In [4]:
# visits on each page (they are in the last column of the second row, we read them manually)
v1_visits = 10283
v2_visits = 2742
v3_visits = 2747
v4_visits = 3180
v5_visits = 2064

#### Click Through rate

Defined as clicks / visits

In [ ]:
# click-through rates
interact_rate = float(v1_clicks / v1_visits)
connect_rate = float(v2_clicks / v2_visits)
learn_rate = float(v3_clicks / v3_visits)
help_rate = float(v4_clicks / v4_visits)
services_rate = float(v5_clicks / v5_visits)

In [ ]:
# CTR from worst to best
rates = pd.Series([interact_rate, connect_rate, learn_rate, help_rate, services_rate])
names = pd.Series(["Interact", "Connect", "Learn", "Help", "Services"])

ctr_df = pd.DataFrame({"rates":rates, "names":names}).sort_values("rates")
ctr_df.sort_values("rates", ascending=False)

,rates,names
4,0.021802,Services
1,0.019329,Connect
3,0.011950,Help
2,0.007645,Learn
0,0.004084,Interact


#### Contingency table

For observed values. We note clicks and no-clicks (defined as visits - clicks)

In [ ]:
# no-clicks
v1_noclick = v1_visits - v1_clicks
v2_noclick = v2_visits - v2_clicks
v3_noclick = v3_visits - v3_clicks
v4_noclick = v4_visits - v4_clicks
v5_noclick = v5_visits - v5_clicks

In [ ]:
# contingency table as a pd.DataFrame creation
clicks = pd.Series([v1_clicks, v2_clicks, v3_clicks, v4_clicks, v5_clicks])
noclicks = pd.Series([v1_noclick, v2_noclick, v3_noclick, v4_noclick, v5_noclick])

observed = pd.DataFrame(data = [clicks, noclicks])
observed.columns = ["Interact", "Connect", "Learn", "Help", "Services"]
observed.index = ["Click", "No-click"]

observed

,Interact,Connect,Learn,Help,Services
Click,42,53,21,38,45
No-click,10241,2689,2726,3142,2019


## Scipy approach

In [ ]:
from scipy import stats
chisq, pvalue, df, expected = stats.chi2_contingency(observed)
print("p value:", pvalue)

p value: 4.852334301093838e-20


In [ ]:
chisq

96.7432353798328

In [ ]:
pd.DataFrame(expected)

,0,1,2,3,4
0,97.36948,25.963932,26.011277,30.111344,19.543967
1,10185.63052,2716.036068,2720.988723,3149.888656,2044.456033


## How do we decide who's the winner?

Read about [Post Hoc Tests](https://alanarnholt.github.io/PDS-Bookdown2/post-hoc-tests-1.html) and find out whether we can declare a clear winner!

In [ ]:
ctr_df.sort_values("rates", ascending=False)

,rates,names
4,0.021802,Services
1,0.019329,Connect
3,0.011950,Help
2,0.007645,Learn
0,0.004084,Interact


We have 10 possible dual tests to perform:
* Interact - Learn
* Interact - Help
* Interact - Connect
* Interact - Services
* Learn - Help
* Learn - Connect
* Learn - Services
* Help - Connect
* Help - Services
* Connect - Services

In [ ]:
alpha = 0.1
possible_combinations = 10
alpha_post_hoc = alpha / possible_combinations
np.round(alpha_post_hoc, 4)

0.01

Let's do the 10 pair-wise tests, and pay close attention to the best performing version:

In [ ]:
# interact vs connect
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Interact", "Connect"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

2.2250331654688293e-16
True


In [ ]:
# interact vs learn
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Interact", "Learn"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

0.025419824342152637
False


In [ ]:
# interact vs help
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Interact", "Help"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

9.03599988558687e-07
True


In [ ]:
# connect vs learn
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Connect", "Learn"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

0.00027678881264505827
True


In [ ]:
# connect vs help
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Connect", "Help"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

0.02808815288948292
False


In [ ]:
# learn vs help
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Learn", "Help"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

0.12512753088691322
False


In [ ]:
# services vs interact
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Interact", "Services"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

5.719451224375125e-18
True


In [ ]:
# services vs learn
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Learn", "Services"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

5.0540996583731365e-05
True


In [ ]:
# services vs help
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Help", "Services"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

0.007370912499282061
True


In [ ]:
# services vs connect
chisq, pvalue, df, expected = stats.chi2_contingency(observed.loc[:, ["Connect", "Services"]])
print(pvalue)
print(pvalue < alpha_post_hoc)

0.6188771123975272
False


The difference between Services and Help is statistically significant, but the difference between Services and Connect is not. 

To decide the winner, we can:

- Look at other metrics besides CTR.
- Refer to the qualitative research.
- Ask opinions to subject-matter experts.
- Redesign the experiment and run it again.